In [2]:
#%load_ext autoreload
#%autoreload 2

#import warnings
#warnings.filterwarnings("ignore")

In [3]:
#from source.utils import *
from source.preprocess import *
import pickle
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict
from tqdm import tqdm_notebook as tqdm

In [4]:
path = "../../Desktop/DLC_social_1/"

# Set up and design the project

In [5]:
with open('{}DLC_social_1_exp_conditions.pickle'.format(path), 'rb') as handle:
    Treatment_dict = pickle.load(handle)

In [6]:
#Which angles to compute?
bp_dict = {'B_Nose':['B_Left_ear','B_Right_ear'],
          'B_Left_ear':['B_Nose','B_Right_ear','B_Center','B_Left_flank'],
          'B_Right_ear':['B_Nose','B_Left_ear','B_Center','B_Right_flank'],
          'B_Center':['B_Left_ear','B_Right_ear','B_Left_flank','B_Right_flank','B_Tail_base'],
          'B_Left_flank':['B_Left_ear','B_Center','B_Tail_base'],
          'B_Right_flank':['B_Right_ear','B_Center','B_Tail_base'],
          'B_Tail_base':['B_Center','B_Left_flank','B_Right_flank']}

In [7]:
%%time
DLC_social_1 = project(path='../../Desktop/DLC_social_1/',#Path where to find the required files
                   smooth_alpha=0.85,                    #Alpha value for exponentially weighted smoothing
                   distances=['B_Center','B_Nose','B_Left_ear','B_Right_ear','B_Left_flank',
                              'B_Right_flank','B_Tail_base'],
                   ego=False,
                   angles=True,
                   connectivity=bp_dict,
                   arena='circular',                  #Type of arena used in the experiments
                   arena_dims=[380],                  #Dimensions of the arena. Just one if it's circular
                   video_format='.mp4',
                   table_format='.h5',
                   exp_conditions=Treatment_dict)

CPU times: user 2.71 s, sys: 847 ms, total: 3.56 s
Wall time: 1.16 s


# Run project

In [8]:
%%time
DLC_social_1_coords = DLC_social_1.run(verbose=True)
print(DLC_social_1_coords)
type(DLC_social_1_coords)

Loading trajectories...
Smoothing trajectories...


/Users/lucas_miranda/PycharmProjects/deepof/source/utils.py:105: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "smooth_mult_trajectory" failed type inference due to: Invalid use of Function(<built-in function array>) with argument(s) of type(s): (list(array(float64, 1d, A)))
 * parameterized
In definition 0:
    TypingError: array(float64, 1d, A) not allowed in a homogeneous sequence
    raised from /Users/lucas_miranda/opt/anaconda3/envs/Machine_Learning/lib/python3.6/site-packages/numba/typing/npydecl.py:472
In definition 1:
    TypingError: array(float64, 1d, A) not allowed in a homogeneous sequence
    raised from /Users/lucas_miranda/opt/anaconda3/envs/Machine_Learning/lib/python3.6/site-packages/numba/typing/npydecl.py:472
This error is usually caused by passing an argument of a type that is unsupported by the named function.
[1] During: resolving callee type: Function(<built-in function array>)
[2] During: typing of call at /

Computing distances...
Computing angles...
Done!
Coordinates of 47 videos across 4 conditions
CPU times: user 10.2 s, sys: 591 ms, total: 10.8 s
Wall time: 14.2 s


source.preprocess.coordinates

# Generate coords

In [9]:
%%time
ptest = DLC_social_1_coords.get_coords(center=True, polar=False, speed=0, length='00:10:00')
ptest._type

CPU times: user 839 ms, sys: 68.7 ms, total: 908 ms
Wall time: 873 ms


'coords'

In [10]:
%%time
dtest = DLC_social_1_coords.get_distances(speed=0, length='00:10:00')
dtest._type

CPU times: user 556 ms, sys: 397 ms, total: 953 ms
Wall time: 977 ms


'dists'

In [11]:
%%time
atest = DLC_social_1_coords.get_angles(degrees=True, speed=0, length='00:10:00')
atest._type

CPU times: user 145 ms, sys: 90.4 ms, total: 235 ms
Wall time: 241 ms


'angles'

# Visualization playground

In [12]:
#ptest.plot_heatmaps(['B_Center', 'W_Center'], i=1)

In [13]:
#Plot animation of trajectory over time with different smoothings
#plt.plot(ptest['Day2Test13DLC']['B_Center'].iloc[:5000]['x'],
#         ptest['Day2Test13DLC']['B_Center'].iloc[:5000]['y'], label='alpha=0.85')

#plt.xlabel('x')
#plt.ylabel('y')
#plt.title('Mouse Center Trajectory using different exponential smoothings')
#plt.legend()
#plt.show()

# Dimensionality reduction playground

In [14]:
#pca = ptest.pca(4, 1000)

In [15]:
#plt.scatter(*pca[0].T)
#plt.show()

# Preprocessing playground

In [16]:
mtest = merge_tables(DLC_social_1_coords.get_coords(center=True, polar=True, length='00:10:00'))#,
#                      DLC_social_1_coords.get_distances(speed=0, length='00:10:00'),
#                      DLC_social_1_coords.get_angles(degrees=True, speed=0, length='00:10:00'))

In [17]:
#pptest = mtest.preprocess(window_size=51, filter='gaussian', sigma=10, shift=20)

In [18]:
pttest = mtest.preprocess(window_size=51, window_step=10, filter=None)
pttest.shape

(70500, 51, 28)

In [19]:
#plt.plot(pttest[2,:,2], label='normal')
#plt.plot(pptest[2,:,2], label='gaussian')
#plt.legend()
#plt.show()

# Trained models playground

### Seq 2 seq Variational Auto Encoder

In [20]:
from datetime import datetime
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.constraints import UnitNorm
from tensorflow.keras.layers import Bidirectional, Dense, Dropout
from tensorflow.keras.layers import Lambda, LSTM
from tensorflow.keras.layers import RepeatVector, TimeDistributed
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam
from source.model_utils import *
import keras as k
import tensorflow as tf

Using TensorFlow backend.


In [21]:
NAME = 'Baseline_AE'
log_dir = os.path.abspath(
    "logs/fit/{}_{}".format(NAME, datetime.now().strftime("%Y%m%d-%H%M%S"))
)
tensorboard_callback = k.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [22]:
from source.models import SEQ_2_SEQ_AE, SEQ_2_SEQ_VAE

In [23]:
encoder, decoder, ae = SEQ_2_SEQ_AE(pttest.shape).build()

In [24]:
encoder, generator, vae = SEQ_2_SEQ_VAE(pttest.shape).build()

In [25]:
pttrain = pttest[:-1500]
pttest  = pttest[-1500:]

In [ ]:
#tf.config.experimental_run_functions_eagerly(False)
history = ae.fit(pttrain, pttrain, epochs=50, batch_size=256, verbose=1, validation_data=(pttest, pttest),
                 callbacks=[tensorboard_callback])

Train on 69000 samples, validate on 1500 samples
Epoch 1/50
